# Задачи

1. Вы работаете в компании социальной сети, которая хочет проанализировать группы пользователей, которые часто взаимодействуют друг с другом. На платформе пользователи могут фолловить друг друга. Компания хочет идентифицировать кластеры пользователей, которые сильно связаны друг с другом.
Помогите компании решить эту задачу, реализовав функцию find_social_clusters:
    -  Функция принимает количество пользователей (n) и список отношений между ними
    -  Возвращает двумерный список групп сильной связанности
    -  Группы должны быть отсортированы по размеру в порядке убывания
    -  В каждой группе идентификаторы пользователей должны быть отсортированы в порядке возрастания

По факту нам нужно лишь настроить правильно отсортировать компоненты сильной связности и аккуратно их вывести. Весь остальной код берём из семинарского ноутбука.

In [10]:
from collections import defaultdict

class Graph:
    def __init__(self):
        # Initialize the graph using defaultdict to store adjacency lists
        self.graph = defaultdict(list)
        self.vertices = set()

    def add_edge(self, u, v):
        """Add a directed edge from vertex u to vertex v"""
        self.graph[u].append(v)
        self.vertices.add(u)
        self.vertices.add(v)

    def transpose(self):
        """Create a transpose graph by reversing all edges"""
        g_transpose = Graph()
        for u in self.graph:
            # For each edge u->v, add edge v->u in transposed graph
            for v in self.graph[u]:
                g_transpose.add_edge(v, u)
        return g_transpose

    def dfs_first_pass(self, vertex, visited, finishing_times):
        """First DFS pass to compute finishing times"""
        visited[vertex] = True

        # Recursively visit all adjacent vertices
        for adj_vertex in self.graph[vertex]:
            if not visited[adj_vertex]:
                self.dfs_first_pass(adj_vertex, visited, finishing_times)

        # Add vertex to finishing_times after exploring all its neighbors
        finishing_times.append(vertex)

    def dfs_second_pass(self, vertex, visited, scc):
        """Second DFS pass to find SCCs"""
        visited[vertex] = True
        scc.append(vertex)

        # Recursively visit all adjacent vertices
        for adj_vertex in self.graph[vertex]:
            if not visited[adj_vertex]:
                self.dfs_second_pass(adj_vertex, visited, scc)

    def find_sccs(self):
        """Main function to find strongly connected components"""
        # Step 1: First DFS pass on original graph
        visited = {vertex: False for vertex in self.vertices}
        finishing_times = []

        # Process all vertices in first DFS pass
        for vertex in self.vertices:
            if not visited[vertex]:
                self.dfs_first_pass(vertex, visited, finishing_times)

        # Step 2: Create transpose graph
        transposed_graph = self.transpose()

        # Step 3: Second DFS pass on transposed graph
        visited = {vertex: False for vertex in self.vertices}
        sccs = []

        # Process vertices in order of decreasing finishing time
        for vertex in reversed(finishing_times):
            if not visited[vertex]:
                current_scc = []
                transposed_graph.dfs_second_pass(vertex, visited, current_scc)
                sccs.append(current_scc)

        return sccs


def find_social_clusters(n, edges):
    g = Graph()

    for u, v in edges:
        g.add_edge(u, v)

    sccs = g.find_sccs()
    mas = []
    print("Strongly Connected Components:")
    for i, scc in enumerate(sccs, 1):
        mas.append(scc)

    mas.sort(key = lambda x: len(x), reverse = True)
    for i in mas:
        i.sort()

    #Этот кусок добавляет в итоговый массив людей, у которых нет сильных связей
    mas1 = [] #Вспомогательный массив
    for i in mas:
        for j in i:
            mas1.append(j)
    for i in range(1, n+1):
        if i not in mas1:
            mas.append([i])

    print(mas)


if __name__ == "__main__":
    n = int(input('Количество пользователей:'))  # Количество пользователей

    # Add edges to create a graph with multiple SCCs
    edges = [(1, 2), (2, 3), (3, 1),  # First SCC: 1-2-3
             (4, 5), (5, 6), (6, 4),  # Second SCC: 4-5-6
             (3, 4), (5, 7)]  # Additional edges

    find_social_clusters(n, edges)

Количество пользователей: 10


Strongly Connected Components:
[[1, 2, 3], [4, 5, 6], [7], [8], [9], [10]]


2. На шахматной доске n × n в некоторой клетке расположен конь. Для каждой клетки найдите минимальное число шагов коня для её достижения. Асимптотика: O(n2).

In [9]:
from collections import deque

def horse(n, x_0, y_0):

    distances = [[-1] * n for _ in range(n)]
    distances[x_0][y_0] = 0
    queue = deque([(x_0, y_0)])

    #На сколько может пойти конь за один ход
    moves = [
        (-2, -1), (-2, 1), (-1, -2), (-1, 2),
        (1, -2), (1, 2), (2, -1), (2, 1)
    ]

    while queue:
        x, y = queue.popleft()

        for dr, dc in moves:
            new_x = x + dr
            new_y = y + dc

            if 0 <= new_x < n and 0 <= new_y < n and distances[new_x][new_y] == -1:
                distances[new_x][new_y] = distances[x][y] + 1
                queue.append((new_x, new_y))

    return distances

n = int(input("Размер доски(одно число): "))
x_0 = int(input("Начальная координата коня по горизонтали: "))
y_0 = int(input("Начальная координата коня по вертикали: "))
result = horse(n, x_0, y_0)

for row in result:
    print(row)


Размер доски(одно число):  10
Начальная координата коня по горизонтали:  3
Начальная координата коня по вертикали:  3


[2, 3, 2, 3, 2, 3, 2, 3, 4, 3]
[3, 4, 1, 2, 1, 4, 3, 2, 3, 4]
[2, 1, 2, 3, 2, 1, 2, 3, 4, 3]
[3, 2, 3, 0, 3, 2, 3, 2, 3, 4]
[2, 1, 2, 3, 2, 1, 2, 3, 4, 3]
[3, 4, 1, 2, 1, 4, 3, 2, 3, 4]
[2, 3, 2, 3, 2, 3, 2, 3, 4, 3]
[3, 2, 3, 2, 3, 2, 3, 4, 3, 4]
[4, 3, 4, 3, 4, 3, 4, 3, 4, 5]
[3, 4, 3, 4, 3, 4, 3, 4, 5, 4]
